In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import SimpleRNN

from keras.models import model_from_json
import numpy as np
import os
np.random.seed(10)

Using TensorFlow backend.


In [2]:
import re
re_tag = re.compile(r'<[^>]+>')

def rm_tags(text):
    return re_tag.sub('', text.rstrip())

In [3]:
def ps(fn,fv=''):
    print (fn, fv)


In [4]:
import os
## type => train / test
## indir => 1, 2, 4, 5, 6 , 7, 8
def read_ref_files(typedir, indir, indirmax=9, readmax=100000):
    path = "data/aclImdb/"
    file_list=[]
    labels_list = []
    
    thisdirpath = '%s/%s' % (typedir, indir)
    for f in os.listdir(thisdirpath)[0:readmax]:
        file_list+= [thisdirpath+os.sep+f]
            
    #print('read',thisdirpath, 'files:',len(file_list))
       
    all_labels = ([indir] * len(file_list)) 
    
    all_texts  = []
    
    for fi in file_list:
        with open(fi,encoding='utf8') as file_input:
            all_texts += [rm_tags(" ".join(file_input.readlines()))]
    
    #print('len-all_labels', len(all_labels), 'len-all_texts', len(all_texts))
    return all_labels,all_texts

In [5]:
train_label, train_text = [], []
for x in range(1, 9):
    ##tmp_label, tmp_text = read_ref_files("train", x, readmax=5000)
    tmp_label, tmp_text = read_ref_files("train", x)
    train_label += tmp_label
    train_text += tmp_text

In [6]:
#print(train_text)
#print(train_label)

In [7]:
test_label, test_text = [], []
for x in range(1, 9):
    ##tmp_label, tmp_text = read_ref_files("test", x, readmax=5000)
    tmp_label, tmp_text = read_ref_files("test", x)
    test_label += tmp_label
    test_text += tmp_text

In [8]:
token = Tokenizer(num_words=30000)
token.fit_on_texts(train_text)

In [9]:
##改成數字list
train_text_numlst = token.texts_to_sequences(train_text)
test_text_numlst  = token.texts_to_sequences(test_text)

In [10]:
##改成數字list長度100
train_text_numseq = sequence.pad_sequences(train_text_numlst, maxlen=100)
test_text_numseq  = sequence.pad_sequences(test_text_numlst,  maxlen=100)

In [11]:
##1 => [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]
##2 => [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]
train_label_digitlst = np_utils.to_categorical(train_label)
test_label_digitlst = np_utils.to_categorical(test_label)

In [12]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation,Flatten
from keras.layers.embeddings import Embedding

In [13]:
model = Sequential()

In [14]:
model.add(Embedding(output_dim=32,
                    input_dim=30000, 
                    input_length=100))
model.add(Dropout(0.2))

In [15]:
model.add(SimpleRNN(units=16))

In [16]:
model.add(Dense(units=256,
                activation='relu' ))
model.add(Dropout(0.2))

In [17]:
model.add(Dense(units=9,
                activation='sigmoid' ))

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 32)           960000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 32)           0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 16)                784       
_________________________________________________________________
dense_1 (Dense)              (None, 256)               4352      
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 2313      
Total params: 967,449
Trainable params: 967,449
Non-trainable params: 0
_________________________________________________________________


# 訓練模型

In [19]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [20]:
##訓練損失(loss) 準確率(acc) 驗證損失(val_loss) 驗證準確率(val_acc)
train_history =model.fit(train_text_numseq, train_label_digitlst, batch_size=100, 
                         epochs=10,verbose=2,
                         validation_split=0.2)

Train on 220694 samples, validate on 55174 samples
Epoch 1/10
 - 138s - loss: 0.1036 - acc: 0.9534 - val_loss: 0.9705 - val_acc: 0.8568
Epoch 2/10
 - 138s - loss: 0.0650 - acc: 0.9658 - val_loss: 1.0716 - val_acc: 0.8634
Epoch 3/10
 - 138s - loss: 0.0606 - acc: 0.9676 - val_loss: 1.1724 - val_acc: 0.8675
Epoch 4/10
 - 138s - loss: 0.0585 - acc: 0.9684 - val_loss: 1.3150 - val_acc: 0.8606
Epoch 5/10
 - 140s - loss: 0.0570 - acc: 0.9688 - val_loss: 1.3040 - val_acc: 0.8620
Epoch 6/10
 - 159s - loss: 0.0559 - acc: 0.9693 - val_loss: 1.3732 - val_acc: 0.8651
Epoch 7/10
 - 211s - loss: 0.0551 - acc: 0.9695 - val_loss: 1.3982 - val_acc: 0.8509
Epoch 8/10
 - 201s - loss: 0.0545 - acc: 0.9696 - val_loss: 1.4924 - val_acc: 0.8513
Epoch 9/10
 - 173s - loss: 0.0541 - acc: 0.9697 - val_loss: 1.4926 - val_acc: 0.8640
Epoch 10/10
 - 136s - loss: 0.0537 - acc: 0.9698 - val_loss: 1.5686 - val_acc: 0.8325


In [21]:
scores = model.evaluate(test_text_numseq, test_label_digitlst, verbose=1)

81500/81500 [==============================] - 30s 370us/step


In [22]:
scores[1]

0.93929787268960407

In [23]:
prediction=model.predict_classes(test_text_numseq)

81500/81500 [==============================] - 31s 375us/step


In [24]:
prediction, len(prediction)

(array([1, 1, 1, ..., 3, 3, 3]), 81500)

In [25]:
def prediction_view(predlst, datalst, ind):
    typedic = {1:'作者', 2:'年分', 3:'題名', 4:'出處', 5:'出版地', 6:'出版者', 7:'卷期', 8:'頁次'}
    print('預測:%s' % typedic.get(predlst[ind]))
    print('資料:%s' % datalst[ind])

In [26]:
print('Train data:%s' % len(train_label_digitlst))

Train data:275868


In [34]:
import random
runed = 1
runmax = 5
while runed <= runmax:
    runint = random.randint(0, len(prediction)-1)
    prediction_view(prediction, test_text, runint)
    runed += 1
    print('index:%s' % runint)
    print('---------------------------')

預測:題名
資料:Group building for successful inclusion program
index:28667
---------------------------
預測:題名
資料:475-486
index:74867
---------------------------
預測:出處
資料:112-115
index:75262
---------------------------
預測:題名
資料:X and R charts for skewed populations
index:37515
---------------------------
預測:題名
資料:吳振麟
index:12214
---------------------------


In [35]:
model_json = model.to_json()
import h5py
with open("SaveModel/keras_reference_RNN_dim30000_outdim64_leru.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("SaveModel/keras_reference_RNN_dim30000_outdim64_leru.h5")
print("Saved model to disk")

Saved model to disk


In [36]:
##save tokenizer
import pickle
pickle.dump(token, open('SaveModel/keras_reference_RNN_dim30000_outdim64_leru.npy', 'wb'))

In [37]:
##load tokenizer
import pickle
load_token = pickle.load(open('SaveModel/keras_reference_RNN_dim30000_outdim64_leru.npy', 'rb'))
print('token len', load_token.document_count)
##print(load_token.word_index)

token len 275868


In [38]:
json_file = open('SaveModel/keras_reference_RNN_dim30000_outdim64_leru.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("SaveModel/keras_reference_RNN_dim30000_outdim64_leru.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
print('token len', load_token.document_count)

Loaded model from disk
token len 275868


In [42]:
##從module & token 中判斷該資料類型
def predict_data(loaded_model, load_token, input_text):
    #load_token.fit_on_texts([input_text])
    print('token len', load_token.document_count)
    input_text_numlst = load_token.texts_to_sequences([input_text])
    input_text_seq = sequence.pad_sequences(input_text_numlst, maxlen=100)
    load_prediction=loaded_model.predict_classes(input_text_seq)
    print('load_prediction', load_prediction)
    prediction_view(load_prediction, [input_text], 0)

In [43]:
input_text = '''以雲端運算為基礎建立醫療輔助決策系統-以冠狀動脈心臟病為例'''
input_text = '''心率減速率和連續心率減速率在心肌梗死後猝死患者中的變化及其意義'''
#input_text = '''鍾振文'''
#input_text = '''Competition, compatibility, and standards'''
#input_text = '''Greeve, H. R.'''
##預測:出處
#input_text = '''Journal of Public Administration: Research and Theory'''
##預測:卷期
#input_text = '''69(January)'''
##預測:年分
#input_text = '''1997'''
##預測:卷期
#input_text = '''18(7)'''
##預測:title
#input_text = '''畢祿溪試驗集水區的降水及溪水化學'''
#input_text = '''中華民國統計年鑑'''
#input_text = '''臺北'''
##print(input_text)
predict_data(loaded_model, load_token, input_text)

token len 275868
1/1 [==============================] - 0s 7ms/step
load_prediction [3]
預測:題名
資料:鍾尚恩
